# Training a classifier on GLCM features

In [99]:
import os
import re
import numpy as np
import nibabel as nib
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from pathlib import Path
import pandas as pd
from scipy.stats import ttest_ind
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd

In [7]:
pd.set_option('display.max_columns', None)

## Loading the data

Since we failed to implement the GLCM extractor in the previous Milestone we will load the data from the file uploaded to the website.

In [9]:
data = np.load("slice_glcm1d.npz", allow_pickle=True)

In [10]:
data.files

['slice_features', 'slice_meta', 'features_rankin_idx']

## Data exploration

In [22]:
df_features = pd.DataFrame(data['slice_features'])

In [23]:
df_features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,3912.728918,1.096981e+07,38704.669443,2122.761852,236.130245,0.799733,8.958410,1.280547,153.858318,0.668218,0.655880,0.986222,0.941925,-0.382916,0.874570,0.000520,58.659495,0.149956,3.108626,0.818609,0.276677,117.318989,2.763986,589.723024
1,7511.568847,3.138924e+07,-19143.340098,4128.827862,188.601517,0.912418,8.283005,1.331034,117.985636,0.640403,0.624592,0.988914,0.944994,-0.483568,0.953910,0.000812,80.784854,0.085426,3.834779,0.918409,0.139321,161.569708,3.407020,1079.357345
2,8858.349761,3.493607e+07,-197109.877954,3355.497691,223.661489,0.874257,8.822815,1.381786,143.925152,0.639497,0.624148,0.987052,0.942295,-0.431027,0.919476,0.000775,89.861605,0.186931,3.461811,0.892283,0.405126,179.723210,3.044884,894.789795
3,9830.155057,1.601792e+07,-118166.648120,1794.384095,208.654995,0.789844,7.819523,1.297230,146.325976,0.689305,0.676263,0.988036,0.949242,-0.344585,0.826478,0.000654,97.126247,0.315186,2.823842,0.817015,0.548032,194.252495,2.435366,500.759772
4,9932.448352,2.210879e+07,-151369.529909,1948.802779,209.900402,0.803567,7.611905,1.277180,150.267237,0.705191,0.693021,0.987998,0.950864,-0.350157,0.824300,0.000604,97.455314,0.344902,2.748613,0.826901,0.577190,194.910628,2.368467,539.675795


In [41]:
df_features.shape

(7414, 24)

In [14]:
for element in data['slice_meta']:
    print(element)

['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
['LIDC-IDRI-0001_GT1_1' 1 1 'NoNod']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_2' 3 2 'Malignant']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_3' 3 3 'NoNod']
['LIDC-IDRI-0003_GT1_4' 3 4 'NoNod']
['LIDC-IDRI-0003_GT1_4' 3 4 'NoNod']
['L

In [28]:
df_meta = pd.DataFrame(data['slice_meta'], columns=['filename', 'patient_id', 'nodule_id', 'diagnosis'])


In [29]:
df_meta.head(10)

,filename,patient_id,nodule_id,diagnosis
0,LIDC-IDRI-0001_GT1_1,1,1,Malignant
1,LIDC-IDRI-0001_GT1_1,1,1,Malignant
2,LIDC-IDRI-0001_GT1_1,1,1,Malignant
3,LIDC-IDRI-0001_GT1_1,1,1,Malignant
4,LIDC-IDRI-0001_GT1_1,1,1,Malignant
5,LIDC-IDRI-0001_GT1_1,1,1,Malignant
6,LIDC-IDRI-0001_GT1_1,1,1,Malignant
7,LIDC-IDRI-0001_GT1_1,1,1,Malignant
8,LIDC-IDRI-0001_GT1_1,1,1,NoNod
9,LIDC-IDRI-0003_GT1_2,3,2,Malignant


In [47]:
df_meta['diagnosis'].unique()

array(['Malignant', 'NoNod', 'Benign'], dtype=object)

In [30]:
labels = df_meta['diagnosis']
labels.head()

0    Malignant
1    Malignant
2    Malignant
3    Malignant
4    Malignant
Name: diagnosis, dtype: object

In [40]:
df_meta.shape

(7414, 4)

In [36]:
df_feature_ranking = pd.DataFrame(data['features_rankin_idx'])

In [38]:
df_feature_ranking.head(10)

,0
0,15
1,16
2,21
3,0
4,18
5,2
6,22
7,10
8,20
9,7


In [39]:
df_feature_ranking.shape

(24, 1)

## Getting rid of NoNods

In [42]:
df_binary = df_features[labels != 'NoNod']

In [43]:
df_binary.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,3912.728918,1.096981e+07,38704.669443,2122.761852,236.130245,0.799733,8.958410,1.280547,153.858318,0.668218,0.655880,0.986222,0.941925,-0.382916,0.874570,0.000520,58.659495,0.149956,3.108626,0.818609,0.276677,117.318989,2.763986,589.723024
1,7511.568847,3.138924e+07,-19143.340098,4128.827862,188.601517,0.912418,8.283005,1.331034,117.985636,0.640403,0.624592,0.988914,0.944994,-0.483568,0.953910,0.000812,80.784854,0.085426,3.834779,0.918409,0.139321,161.569708,3.407020,1079.357345
2,8858.349761,3.493607e+07,-197109.877954,3355.497691,223.661489,0.874257,8.822815,1.381786,143.925152,0.639497,0.624148,0.987052,0.942295,-0.431027,0.919476,0.000775,89.861605,0.186931,3.461811,0.892283,0.405126,179.723210,3.044884,894.789795
3,9830.155057,1.601792e+07,-118166.648120,1794.384095,208.654995,0.789844,7.819523,1.297230,146.325976,0.689305,0.676263,0.988036,0.949242,-0.344585,0.826478,0.000654,97.126247,0.315186,2.823842,0.817015,0.548032,194.252495,2.435366,500.759772
4,9932.448352,2.210879e+07,-151369.529909,1948.802779,209.900402,0.803567,7.611905,1.277180,150.267237,0.705191,0.693021,0.987998,0.950864,-0.350157,0.824300,0.000604,97.455314,0.344902,2.748613,0.826901,0.577190,194.910628,2.368467,539.675795


In [45]:
df_binary.shape

(2383, 24)

In [90]:
filt_nonod = df_meta[df_meta['diagnosis'] != 'NoNod']

In [91]:
filt_nonod.shape

(2383, 4)

In [92]:
labels = filt_nonod['diagnosis']

In [59]:
labels.unique()

array(['Malignant', 'Benign'], dtype=object)

## Performing the t-test for feature importance

Is this correct? What if we have feature correlations? 

In [69]:
# 'Malignant', 'NoNod', 'Benign'
class_0 = df_binary[labels == 'Benign']  # Subset where class is 0
class_1 = df_binary[labels == 'Malignant']  # Subset where class is 1

# Perform t-test for each feature
p_values = {col: ttest_ind(class_0[col], class_1[col], equal_var=True).pvalue for col in df_features.columns}

# Convert results to DataFrame
feature_importance = pd.DataFrame.from_dict(p_values, orient='index', columns=['p_value'])

# Sort by significance
feature_importance = feature_importance.sort_values(by='p_value')

print(feature_importance.head(24))  # Features with the smallest p-values

         p_value
15  1.041978e-35
16  6.250837e-08
21  6.250837e-08
0   5.315320e-07
18  1.193866e-05
2   1.818947e-05
22  2.126097e-05
10  2.684689e-04
20  3.112284e-04
7   3.398321e-04
9   7.353698e-04
8   8.400409e-03
17  1.363838e-02
5   5.373388e-02
11  5.518758e-02
4   5.645414e-02
23  7.537756e-02
6   8.396037e-02
12  1.022167e-01
3   1.220338e-01
19  2.335962e-01
1   6.855817e-01
14  7.524653e-01
13  8.305820e-01


#### Check if my t_tests are the same as the ones I loaded

In [70]:
all(df_feature_ranking[0] == feature_importance.index)

True

### Filter out p values > 0.05

In [72]:
important_features = feature_importance[feature_importance < 0.05]

In [76]:
important_features.shape

(24, 1)

In [73]:
important_features.head(24)

,p_value
15,1.041978e-35
16,6.250837e-08
21,6.250837e-08
0,5.315320e-07
18,1.193866e-05
2,1.818947e-05
22,2.126097e-05
10,2.684689e-04
20,3.112284e-04
7,3.398321e-04


In [77]:
important_features = important_features.dropna()
important_features.shape

(13, 1)

In [71]:
df_binary.columns

RangeIndex(start=0, stop=24, step=1)

In [78]:
df_binary_imp = df_binary[important_features.index]

In [79]:
df_binary_imp.columns

Index([15, 16, 21, 0, 18, 2, 22, 10, 20, 7, 9, 8, 17], dtype='int64')

## Training SVM on Stratified k-fold

In [98]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
  # default parameters
metrics = []

for fold, (train_idx, test_idx) in enumerate(skf.split(df_binary_imp, labels)):
    print(f"\n--- Fold {fold + 1} ---")
    X_train, X_test = df_binary_imp.iloc[train_idx], df_binary_imp.iloc[test_idx]
    y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # report = classification_report(y_test, y_pred, digits=4, output_dict=True)
    # metrics.append(report)
    # 
    report = classification_report(y_test, y_pred, output_dict=True)
    print(report)

    # Flatten the dictionary into a DataFrame row
    report_df = pd.DataFrame(report).transpose()
    # report_df['Fold'] = fold  # Add fold number

    metrics.append(report_df)

# Combine all fold reports into one DataFrame
df_metrics = pd.concat(metrics, axis=0).reset_index()

# Compute average metrics across folds (excluding support count)
avg_metrics = df_metrics.groupby('index').mean(numeric_only=True)

print("____________________________________________")
print("average metrics: ")
print(avg_metrics)  # Displays properly formatted averaged classification report


--- Fold 1 ---
{'Benign': {'precision': 0.6666666666666666, 'recall': 0.20408163265306123, 'f1-score': 0.3125, 'support': 49.0}, 'Malignant': {'precision': 0.9155844155844156, 'recall': 0.9883177570093458, 'f1-score': 0.950561797752809, 'support': 428.0}, 'accuracy': 0.9077568134171907, 'macro avg': {'precision': 0.7911255411255411, 'recall': 0.5961996948312035, 'f1-score': 0.6315308988764046, 'support': 477.0}, 'weighted avg': {'precision': 0.8900142485048146, 'recall': 0.9077568134171907, 'f1-score': 0.8850166654888936, 'support': 477.0}}

--- Fold 2 ---
{'Benign': {'precision': 0.5555555555555556, 'recall': 0.10204081632653061, 'f1-score': 0.1724137931034483, 'support': 49.0}, 'Malignant': {'precision': 0.905982905982906, 'recall': 0.9906542056074766, 'f1-score': 0.9464285714285714, 'support': 428.0}, 'accuracy': 0.89937106918239, 'macro avg': {'precision': 0.7307692307692308, 'recall': 0.5463475109670036, 'f1-score': 0.5594211822660098, 'support': 477.0}, 'weighted avg': {'precisi

In [88]:
(0.6666666666666666 + 0.5555555555555556 + 0.7142857142857143 + 0.7777777777777778 + 0.8461538461538461) / 5

0.7120879120879121

average calculation is correct

In my opinion the model performs well, the most important factor from a patient's perspective, ,Did I find the tumors that are actually Malignant"? (Recall) is almost 100%. However, we can improve on the other metrics to reduce the chance of unnecessary bad news for the patients and eliminate / reduce the cost of further diagnoses.

In [93]:
filt_nonod.head()

,filename,patient_id,nodule_id,diagnosis
0,LIDC-IDRI-0001_GT1_1,1,1,Malignant
1,LIDC-IDRI-0001_GT1_1,1,1,Malignant
2,LIDC-IDRI-0001_GT1_1,1,1,Malignant
3,LIDC-IDRI-0001_GT1_1,1,1,Malignant
4,LIDC-IDRI-0001_GT1_1,1,1,Malignant


In [94]:
filt_nonod.shape

(2383, 4)

In [95]:
nodules = filt_nonod['nodule_id']

## Training SVM on Stratified Group K-Fold

In [97]:
cv = StratifiedGroupKFold(n_splits=5)

for fold, (train_idx, test_idx) in enumerate(cv.split(df_binary_imp, labels, nodules)):
    print(f"\n--- Fold {fold + 1} ---")
    X_train, X_test = df_binary_imp.iloc[train_idx], df_binary_imp.iloc[test_idx]
    y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]

    # Feature Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    clf = SVC()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # report = classification_report(y_test, y_pred, digits=4, output_dict=True)
    # metrics.append(report)
    # 
    report = classification_report(y_test, y_pred, output_dict=True)
    print(report)

    # Flatten the dictionary into a DataFrame row
    report_df = pd.DataFrame(report).transpose()
    # report_df['Fold'] = fold  # Add fold number

    metrics.append(report_df)

# Combine all fold reports into one DataFrame
df_metrics = pd.concat(metrics, axis=0).reset_index()

# Compute average metrics across folds (excluding support count)
avg_metrics = df_metrics.groupby('index').mean(numeric_only=True)

print("____________________________________________")
print("average metrics: ")
print(avg_metrics)  # Displays properly formatted averaged classification report


--- Fold 1 ---
{'Benign': {'precision': 0.5714285714285714, 'recall': 0.23931623931623933, 'f1-score': 0.3373493975903614, 'support': 117.0}, 'Malignant': {'precision': 0.9135922330097087, 'recall': 0.9781704781704782, 'f1-score': 0.9447791164658634, 'support': 962.0}, 'accuracy': 0.8980537534754403, 'macro avg': {'precision': 0.74251040221914, 'recall': 0.6087433587433587, 'f1-score': 0.6410642570281124, 'support': 1079.0}, 'weighted avg': {'precision': 0.8764901492238023, 'recall': 0.8980537534754403, 'f1-score': 0.8789132433347848, 'support': 1079.0}}

--- Fold 2 ---
{'Benign': {'precision': 0.75, 'recall': 0.16216216216216217, 'f1-score': 0.26666666666666666, 'support': 37.0}, 'Malignant': {'precision': 0.9460869565217391, 'recall': 0.9963369963369964, 'f1-score': 0.9705619982158786, 'support': 546.0}, 'accuracy': 0.9433962264150944, 'macro avg': {'precision': 0.8480434782608696, 'recall': 0.5792495792495793, 'f1-score': 0.6186143324412726, 'support': 583.0}, 'weighted avg': {'pre

{'Benign': {'precision': 1.0, 'recall': 0.09803921568627451, 'f1-score': 0.17857142857142858, 'support': 51.0}, 'Malignant': {'precision': 0.8553459119496856, 'recall': 1.0, 'f1-score': 0.9220338983050848, 'support': 272.0}, 'accuracy': 0.8575851393188855, 'macro avg': {'precision': 0.9276729559748428, 'recall': 0.5490196078431373, 'f1-score': 0.5503026634382566, 'support': 323.0}, 'weighted avg': {'precision': 0.8781860311155247, 'recall': 0.8575851393188855, 'f1-score': 0.8046450872945075, 'support': 323.0}}

--- Fold 4 ---
{'Benign': {'precision': 1.0, 'recall': 0.75, 'f1-score': 0.8571428571428571, 'support': 4.0}, 'Malignant': {'precision': 0.9949494949494949, 'recall': 1.0, 'f1-score': 0.9974683544303797, 'support': 197.0}, 'accuracy': 0.9950248756218906, 'macro avg': {'precision': 0.9974747474747474, 'recall': 0.875, 'f1-score': 0.9273056057866185, 'support': 201.0}, 'weighted avg': {'precision': 0.995050002512689, 'recall': 0.9950248756218906, 'f1-score': 0.9946758072206777, 's

### Conclusion:

This method is useful when you have grouped data, like medical records where multiple samples belong to the same patient. It prevents data leakage by ensuring that all samples from a single group are either in the training or test set, but never both.

Our results are slightly worse, possibly because of the small ammout of data leakage prevented

## Hyperparameter tuning

In [109]:
# Define parameter grid
param_grid = {
    'svc__C': [0.01, 0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

# param_grid = {
#     'svc__C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization (default=1.0)
#     'svc__kernel': ['linear', 'rbf'],  # Kernel type (default='rbf')
#     'svc__shrinking': [True, False],  # Use shrinking heuristic (default=True)
#     'svc__probability': [True, False],  # Enable probability estimates (default=False)
#     'svc__tol': [1e-3, 1e-4, 1e-5],  # Tolerance for stopping (default=1e-3)
#     'svc__class_weight': [None, 'balanced']  # Handle imbalanced classes (default=None)
# }

metrics = []
cv = StratifiedGroupKFold(n_splits=5)
# cv_inner = StratifiedGroupKFold(n_splits=5)

for fold, (train_idx, test_idx) in enumerate(cv.split(df_binary_imp, labels, nodules)):
    print(f"\n--- Fold {fold + 1} ---")
    X_train, X_test = df_binary_imp.iloc[train_idx], df_binary_imp.iloc[test_idx]
    y_train, y_test = labels.iloc[train_idx], labels.iloc[test_idx]
    groups_train = nodules.iloc[train_idx]

    # Pipeline with scaler and SVC
    pipeline = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
    
    # Grid Search
    clf = GridSearchCV(pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    clf.fit(X_train, y_train, groups=groups_train)
    
    # Evaluate
    y_pred = clf.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    print(report)
    report_df = pd.DataFrame(report).transpose()
    metrics.append(report_df)


--- Fold 1 ---
{'Benign': {'precision': 0.6136363636363636, 'recall': 0.23076923076923078, 'f1-score': 0.33540372670807456, 'support': 117.0}, 'Malignant': {'precision': 0.9130434782608695, 'recall': 0.9823284823284824, 'f1-score': 0.9464196294441662, 'support': 962.0}, 'accuracy': 0.9008341056533827, 'macro avg': {'precision': 0.7633399209486166, 'recall': 0.6065488565488566, 'f1-score': 0.6409116780761204, 'support': 1079.0}, 'weighted avg': {'precision': 0.8805776465545978, 'recall': 0.9008341056533827, 'f1-score': 0.8801648930029032, 'support': 1079.0}}

--- Fold 2 ---
{'Benign': {'precision': 1.0, 'recall': 0.08108108108108109, 'f1-score': 0.15, 'support': 37.0}, 'Malignant': {'precision': 0.9413793103448276, 'recall': 1.0, 'f1-score': 0.9698046181172292, 'support': 546.0}, 'accuracy': 0.9416809605488851, 'macro avg': {'precision': 0.9706896551724138, 'recall': 0.5405405405405406, 'f1-score': 0.5599023090586146, 'support': 583.0}, 'weighted avg': {'precision': 0.9450996628615367,

In [110]:
# Aggregate results
df_metrics = pd.concat(metrics, axis=0).reset_index()
avg_metrics = df_metrics.groupby('index').mean(numeric_only=True)
print("\nAverage Metrics (Grid Search):")
print(avg_metrics)


Average Metrics (Grid Search):
              precision    recall  f1-score     support
index                                                  
Benign         0.922727  0.198645  0.296898   49.000000
Malignant      0.905923  0.996466  0.948050  427.600000
accuracy       0.904527  0.904527  0.904527    0.904527
macro avg      0.914325  0.597555  0.622474  476.600000
weighted avg   0.910999  0.904527  0.872081  476.600000


My accuracy metrics just got worse